# Numerical Integration

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


## Question 1

- a) Derivate the trapezoidal rule from Newton-Cotes ($n=1$)
- b) Derivate the Simpson 1/3 rule from Newton-Cotes ($n=2$)


**Solution:**

Newton-Cotes approximates $f(x)$ py a polynomial of degree $n$:
$$f(x) \approx \sum_{i=0}^n f(x_i) l_i(x) \text{ with } l_i(x) = \prod_{j \neq i}\frac{x - x_j}{x_i - x_j}$$
The approximation of the integral is thus:
$$I \approx \sum_{i=0}^n f(x_i) \underbrace{\int_a^b l_i(x) \,\mathrm{d}x}_{A_i}$$

- a) For $n = 1$ (trapezoidal rule):

$$l_0 = \frac{(x - b)}{-h} \Longrightarrow A_0 = \frac{1}{2h} (b - a)^2 = \frac h 2$$

$$l_1 = \frac{(x - a)}{h} \Longrightarrow A_1 = \frac{1}{2h} (b - a)^2 = \frac h 2$$

$$\Longrightarrow I \approx \frac h 2 \big( f(a) + f(b) \big)$$

- b) For $n = 2$ (Simpson 1/3 rule):

$$l_0 = \frac{\left( x - \frac{a+b}2 \right)(x - b)}{2h^2} \Longrightarrow A_0 = \frac h 3$$

$$l_1 = \frac{(x - a)(x - b)}{-h^2} \Longrightarrow A_1 = \frac {4h} 3$$

$$l_2 = \frac{(x - a)\left( x - \frac{a+b}2 \right)}{2h^2} \Longrightarrow A_2 = \frac h 3$$

$$\Longrightarrow I \approx \frac h 3 \left( f(a) + 4 f \Big(\frac{a+b}2\Big) + f(b) \right)$$

## Question 2

- a) Implement the composite trapezoidal rule
- b) Do the same using a recursive trapzoidal rule
- c) Use both implementation to compute the integral of $\ln(1 + \tan x)$ from $0$ to $\frac \pi 4$.  Which one run faster ?


In [3]:
from time import time

def trapezoid_comp(f, a, b, n):
    h = (b - a) / n
    x = a
    I = 0
    c = 1
    fx = f(x)
    for _ in range(n):
        fxh = f(x + h)
        I += fx + fxh
        x += h
        fx = fxh
        c = c + 1
    print(c)
    return h/2 * I


def trapezoid_rec(f, a, b, k, c=0):
    if k == 1:
        c += 2
        print(c)
        return (f(a) + f(b)) * (b - a) / 2
    n = 2 ** (k - 2)
    h = (b - a) / n
    x = a + h/2
    s = 0
    for _ in range(n):
        s += f(x)
        c += 1
        x += h
    return (trapezoid_rec(f, a, b, k-1,c) + h*s) / 2


def f(x): 
    return np.log(1 + np.tan(x))

t0 = time()
print(trapezoid_comp(f, 0, np.pi / 4, 16))
t1 = time()
print(trapezoid_rec(f, 0, np.pi / 4, 5))
t2 = time()
print("Comp exe time: " + str((t1 - t0) * 1e3) + " ms")
print("Rec exe time: " + str((t2 - t1) * 1e3) + " ms")


17
0.27219826128795027
17
0.2721982612879502
Comp exe time: 0.28204917907714844 ms
Rec exe time: 0.15854835510253906 ms


## Question 3

Evaluate the integral of $\cos(2 \cos^{−1} x)$ from $-1$ to $1$ with Simpson’s 1/3 rule using two, four, and six
panels. Explain the results.

In [4]:
import numpy as np

def simpson_comp(f, a, b, n):
    h = (b - a) / n
    x = a
    I = 0
    for _ in range(0, n, 2):
        I += f(x) + 4*f(x + h) + f(x + 2*h)
        x += 2*h
    return h/3 * I

def g(x): 
    return np.cos(2 * (1 / np.cos(x)))

print(simpson_comp(g, -1, 1, 2))
print(simpson_comp(g, -1, 1, 4))
print(simpson_comp(g, -1, 1, 6))
print(simpson_comp(g, -1, 1, 8))
print(simpson_comp(g, -1, 1, 10))
print(simpson_comp(g, -1, 1, 12))
print(simpson_comp(g, -1, 1, 14))
print(simpson_comp(g, -1, 1, 16))
print(simpson_comp(g, -1, 1, 50))
print(simpson_comp(g, -1, 1, 100))
print(simpson_comp(g, -1, 1, 1000))

-1.1196854554190074
-1.2884087548681498
-1.339188110791998
-1.355011028796083
-1.3607412047898517
-1.3631096493137569
-1.364200885700604
-1.3647503670372387
-1.3655364736135815
-1.3655439592904242
-1.3655444513696322


## Question 4

Evaluate the integral of $\cos(2 \cos^{−1} x)$ from $-1$ to $1$ using Romberg Integration
How many steps are necessary to achieve convergence ?


In [5]:
def romberg(f,n,a,b):
    H = b-a
    R = np.zeros(shape = (n,n))
    R[0,0] = (f(a)+f(b))*((b-a)/2)
    
    for k in range(2,n+1):
        ks = np.arange(1,2**(k-2)+1)
        points = a + (2*(ks)-1)*H/(2**(k-1))
        s = H/(2**(k-1))*sum(f(points))
        R[k-1,0] = R[k-2,0]/2 + s
        
        for k2 in range(2,k+1):
            R[k-1,k2-1] = (4**(k2-1)*R[k-1,k2-2] - R[k-2,k2-2])/(4**(k2-1)-1)
        
        
    #print(R)
    return np.diag(R)

def f(x): return np.cos(2 * (1 / np.cos(x)))

romberg(f,10,-1,1)
    

array([-1.69446902, -1.11968546, -1.29965697, -1.36040029, -1.36551405,
       -1.36554826, -1.3655445 , -1.36554445, -1.36554445, -1.36554445])

## Question 5

Given the data: 

| x | f(x) |
| --- | --- |
| 0.0 | 1.0000 |
| $\frac \pi 4 $ | 0.3431 |
| $\frac \pi 2 $ | 0.2500 |
| $\frac {3\pi} 4$ | 0.3431 |
| $ \pi $ | 1.000 |

- evaluate the integral of $f(x)$ from $0$ to $\pi$ as precisely as possible


**Solutions**

We can use Romberg Integration until k=3.

In [6]:
def f(x):
    if x == 0.0: return 1
    elif x == np.pi/4: return 0.3431
    elif x == np.pi/2: return 0.25
    elif x == 3*np.pi/4: return 0.3431
    elif x == np.pi: return 1
    print("ERROR")
    print(x)
    
f = np.vectorize(f)

romberg(f,3,0,np.pi)

array([3.14159265, 1.57079633, 1.3599047 ])

## Question 6

- Use Gauss-Laguerre quadrature to evaluate the integral of $(1 − x^2)^3 e^{−x}$ from $0$ to $\infty$

**Solutions**

Using the table from the slides, with n=3 -> exact integral because (1-x^2)^3 is a polynomial of degree 2*3+1 or less.

In [17]:
def f(x): return (1-x**2)**3

I = 0.603154*f(0.322548) + 0.357418*f(1.745761) + 0.0388791*f(4.536620) + f(9.395071)*0.000539295
print(I)
I = 0.521756*f(0.263560) + 0.398667*f(1.413403) + 0.0759424*f(3.596426) + 0.00361175*f(7.085810) + 0.0000233670*f(12.640801)
print(I)

I = 0.458964*f(0.222847) + 0.417*f(1.188932) + 0.113373*f(2.992736) + 0.0103992*f(5.775144) + 0.000261017*f(9.837467)+0.000000898548*f(15.982874)
print(I)

-652.9339577765277
-652.9870382069656
-652.9997793173131
